In [97]:
import pandas as pd
import gzip
import numpy as np
import re
import datetime

This notebook is destinated to find the collaborators IDs on each video.


The organization of the notebook: 
1) First taks: functions that will help us find the collaborators IDs, and the dataset needed to find the IDS.
2) Second task: Importing the dataset and applying our functions to it.
3) Third task: Create a new dataframe with collaborators IDs per week (for a join with our other dataset)


A) ***FIRST PART: FUNCTIONS:***

In [124]:
#dataset we will use in subsequest functions to find the ID
df_channels_path = '/Users/selima/Downloads/df_channels_en.tsv'
df_channels = pd.read_csv(df_channels_path, sep ='\t')
#keeping only the channel_id and channel_name
channel_ids_and_names = df_channels[['channel' , 'name_cc']].copy()
channel_ids_and_names.rename(columns={"channel": "channel_id", "name_cc": "channel_name"} , inplace= True)

1) **Function 1: Scrap the youtube links in the description**

First, we will see if there is any of this pattern of links in the description

identified channel links types:
1) youtube.com/channel/youtubecreators
2) youtube.com/@youtubecreators
3) youtube.com/c/YouTubeCreators
4) youtube.com/user/YouTube


We know that for URL 1) and 4) we can find the channel ID wheras in URL 2) and 3) we will find the channel names

In our function, if the link refears to the name of a channel, we append it to a new column in our dataframe called 'channel_name', and if it refears an ID we append it to another one 'channel_id'.

If there is more than a channel mentionned, we take all the names.

In [100]:
# function to extract all youtube channel IDs and channel names
#4.4 seconds to run
def extract_youtube_channels(text):
    # Regular expressions for finding YouTube channel URLs
    channel_id_pattern = r'https?://www\.youtube\.com/(?:channel/|user/)([\w-]+)'
    channel_name_pattern = r'https?://www\.youtube\.com/(?:@|c/)([\w-]+)'

    # Find all links that match the patterns
    channel_ids = re.findall(channel_id_pattern, text)
    channel_names = re.findall(channel_name_pattern, text)

    # Join the IDs and names found (or np.nan if not found)
    channel_id = ', '.join(channel_ids) if channel_ids else pd.NA
    channel_name = ', '.join(channel_names) if channel_names else pd.NA

    return channel_id, channel_name

# apply the function and create two new columns
#data2[['mentionned_channel_ID', 'mentionned_channel_name']] = data2['description'].apply(
    #lambda x: pd.Series(extract_youtube_channels(x)))



Keeping only the channel names and IDs.

2) **Function 2: Filter the IDs obtained. To have a valid ID a) it needs to be an existing ID in our channel_ids_and_names dataset 2) it needs to different from the ID of the creator of the video.**

In [125]:
#convert to unique ids to set for faster loop
unique_ids_set = set(channel_ids_and_names.channel_id.unique())

In [126]:
def filter_ids_by_existance_and_not_self_mentioning(df, unique_ids_set):
    # Define a vectorized function
    def process_entry(mentioned_ids, channel_id):
        if pd.isna(mentioned_ids):
            return pd.NA
        ids = [id.strip() for id in mentioned_ids.split(',')]
        valid_ids = [id for id in ids if id in unique_ids_set and id != channel_id]
        return ','.join(valid_ids) if valid_ids else pd.NA

    # Split, process, and rejoin without using apply row-wise
    filtered = [process_entry(m_id, c_id) for m_id, c_id in zip(df['mentionned_channel_ID'], df['channel_id'])]
    
    return filtered

3) **Function 3: get the channel IDs from channel names we have**

We need to a) check that the names are valid (exist in our dataframe) b) add the IDs corresponding to valid names

In [128]:
# Create a dictionary for faster lookup
channel_name_to_id = pd.Series(channel_ids_and_names.channel_id.values, index=channel_ids_and_names.channel_name.str.lower()).to_dict()

In [ ]:
def get_channel_ids_from_valid_names(mentionned_named, channel_name_to_id):
    def process_entry(entry):
        # Handle missing values
        if pd.isna(entry):
            return pd.NA
        names = str(entry).split(',')
        # Lookup each name in the channel_name_to_id dictionary
        valid_ids = [channel_name_to_id.get(name.strip().lower()) for name in names if name.strip().lower() in channel_name_to_id]
        # Filter out None values in case a name wasn't found in the dictionary
        valid_ids = [id for id in valid_ids if id is not None]
        return ','.join(valid_ids) if valid_ids else pd.NA

    # list comprehension instead of apply for potentially better performance
    return [process_entry(entry) for entry in mentionned_named]

Now : check that the id we got isn't 1) of the video creator , 2) doesn't appear in filtered_mentionned_channel_ID

4) **Function 4: Check the validity of the IDs we got**

Checks done a) ids_from_valid_names are not the same as the channel_id b) the id of the channel name doesn't appear in our filtered_mentionned_channel_ID 

In [107]:
def remove_redundant_ids(row):
    # split the ids into a list, remove any whitespace, and filter out any empty strings
    valid_ids = list(filter(None, [x.strip() for x in str(row['ids_from_valid_names']).split(',')]))
    mentioned_ids = set(filter(None, [x.strip() for x in str(row['filtered_mentionned_channel_ID']).split(',')]))
    
    # remove the channel_id from the valid_ids if it's present
    valid_ids = [id for id in valid_ids if id != row['channel_id']]
    
    # remove any id from valid_ids if it's already present in mentioned_ids
    valid_ids = [id for id in valid_ids if id not in mentioned_ids]
    
    # join the valid ids back into a string
    return ','.join(valid_ids) if valid_ids else pd.NA

5) **Function 5: concatenate both columns of IDs we got to have the final colaborator IDs**

In [184]:
def concatenate_columns(row):
    val1 = row['filtered_mentionned_channel_ID']
    val2 = row['clean_ids_from_valid_names']
    
    # Check for both np.nan and '<NA>' string
    if pd.isna(val1) and (pd.isna(val2)):
        return pd.NA
    elif pd.isna(val1):
        return val2
    elif pd.isna(val2):
        return val1
    else:
        return f"{val1},{val2}"

6) **Function 6: group all our functions into one**

In [185]:
def preprocessing_chunk(chunk):
    # work on a copy to avoid the error
    chunk = chunk.copy()
    
    #function 1
    chunk[['mentionned_channel_ID', 'mentionned_channel_name']] = chunk['description'].apply(
        lambda x: pd.Series(extract_youtube_channels(x)))
    
    # drop these columns as we don't need them, work on a less heavy dataset
    chunk.drop(['tags', 'description', 'title'], axis=1, inplace=True)
    
    #function 2
    chunk['filtered_mentionned_channel_ID'] = filter_ids_by_existance_and_not_self_mentioning(
        chunk[['mentionned_channel_ID', 'channel_id']], unique_ids_set)
    
    #function 3
    chunk['ids_from_valid_names'] = get_channel_ids_from_valid_names(chunk['mentionned_channel_name'], channel_name_to_id)
    
    #function 4
    chunk['clean_ids_from_valid_names'] = chunk.apply(remove_redundant_ids, axis=1)

    #function 5
    chunk['final_colaborator_ids'] = chunk.apply(concatenate_columns, axis=1)
    
    #dropping the columns we don't need anymore
    preprocessed_chunk = chunk.drop(['mentionned_channel_ID', 'mentionned_channel_name',
                                     'filtered_mentionned_channel_ID', 'ids_from_valid_names',
                                     'clean_ids_from_valid_names'], axis=1)
    
    
    return preprocessed_chunk

B) ***SECOND PART: importing the dataset and applying the preprocessing steps***

In [186]:
# empty list to store the processed chunks
final_dataframe_list = []

# read the dataset in chunks and preprocess each chunk
with gzip.open(data_path, 'rt', encoding='utf-8') as file:
    for chunk in pd.read_json(file, lines=True, chunksize=chunk_size, convert_dates=['upload_date']):
        # filter the chunk
        filtered_chunk = chunk[
            (chunk['categories'] == 'Gaming') & 
            (chunk['upload_date'] > pd.Timestamp('2016-01-01'))
        ]
        # preprocess the filtered chunk
        preprocessed_chunk = preprocessing_chunk(filtered_chunk)
        # append the preprocessed chunk to the list
        final_dataframe_list.append(preprocessed_chunk)

# concatenate all the preprocessed chunks into a single DataFrame
final_dataframe = pd.concat(final_dataframe_list, ignore_index=True)

Do we have sufficient data for collaborations?

In [140]:
final_dataframe.isnull().sum()

categories                     0
channel_id                     0
crawl_date                     0
dislike_count              81608
display_id                     0
duration                       0
like_count                 81608
upload_date                    0
view_count                     1
final_colaborator_ids    8881049
dtype: int64

In [138]:
nb_not_nan_collaborators = final_dataframe.final_colaborator_ids.dropna().count() / final_dataframe.shape[0]
print('We have %0.2f percent of rows where the collaborator ID is not NaN. This represents %i out of %i rows.' % (nb_not_nan_collaborators * 100, final_dataframe.final_colaborator_ids.dropna().count(), final_dataframe.shape[0]))

We have 3.11 percent of rows where the collaborator ID is not NaN. This represents 285228 out of 9166277 rows.


Let's export our dataframe

In [119]:
final_dataframe.to_csv('/Users/selima/Desktop/dataframe_collaborations_videos.csv', index=False)

**THIRD PART: Create a dataframe for collaborators name per week**

We will now create another dataframe, which will contain 3 columns: 1) the channel id of the video creator 2) the week of the year the video was created 3) collaborator ids for this week

In [168]:
# function to concatenate collaborator IDs, handling NaN values
def concatenate_ids(series):
    # check if all values are NaN, if so, return NaN
    if (series.isna().all()) :
        return np.nan # represent missing data
    # otherwise, concatenate non-NaN values
    return ','.join(series.dropna().astype(str))


In [ ]:
#add a column Year-Week to our final_dataframe
final_dataframe['Year-Week'] = final_dataframe['upload_date'].dt.strftime('%Y-%U')

In [169]:
# Group by 'Year-Week' and 'channel_id', and concatenate the final_collaborator_ids
grouped = final_dataframe.groupby(['Year-Week', 'channel_id'])

# creating a new dataframe with the the new column
collaborators_per_week = grouped.agg({
    'final_colaborator_ids': concatenate_ids
}).reset_index()

# rename the new column
collaborators_per_week.rename(columns={'final_colaborator_ids': 'collaborators_in_week'}, inplace=True)


In [172]:
def replace_empty_with_nan(series):
    # Replace empty strings with NaN
    return series.replace('', pd.NA)

In [174]:
collaborators_per_week['collaborators_in_week'] = replace_empty_with_nan(collaborators_per_week['collaborators_in_week'])

In [ ]:
def concatenate_ids(series):
    # Convert all missing value indicators to pd.NA
    series = series.replace({np.nan: pd.NA})
    
    # Check if all values are pd.NA, if so, return pd.NA
    if series.isna().all():
        return pd.NA
    else:
        # Concatenate non-missing values
        return ','.join(series.dropna().astype(str))

In [179]:
collaborators_per_week.collaborators_in_week.dropna()

40                             UCuAUR88qBEk1rNnk4XD7L1A,<NA>
149        UCXC3bB5hQzpRT1fFS0vcWcQ,UCNjdFawISR3ybZcye9tn...
186                            UCuGzr7zMBxGcvBCNhl409sg,<NA>
232        UCp9AkWp4jfrEyZKhOTo5rbA,UClw5UTugvHO-VL7n-Iax...
243                            UCKFtM4-dl0mDXwmfZlkSWoQ,<NA>
                                 ...                        
1872707    UCOmy8wuTpC95lefU5d1dt2Q,<NA>,UCOmy8wuTpC95lef...
1872797                        UCTjMoP3OH9WJA3Td4jd7xCA,<NA>
1872804    UCv23U-Dk9kg65Ykmz2wDnpQ,<NA>,UCv23U-Dk9kg65Yk...
1872808    UCKhtonFjPN0GJKkJW6iFXng,<NA>,UCKhtonFjPN0GJKk...
1872813    UCzW9IT5ylJ87DLcorSpM2Eg,UC4wUSUO1aZ_NyibCqIjp...
Name: collaborators_in_week, Length: 101109, dtype: object

In [ ]:
collaborators_per_week.to_csv('/Users/selima/Desktop/collaborators_per_week.csv', index=False)